In [20]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
import random
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from PIL import Image
import uuid

In [21]:
# PREPROCESS: load local image or user-upload image, save to processing folder

In [22]:
img_source_dir = './images/train'
img_name_list = os.listdir(img_source_dir)
random_image_name = random.choice(img_name_list)
full_image_path = img_source_dir + '/' + random_image_name
target_source_dir = './images/waiting_for_classification'
if os.path.isdir(target_source_dir) == False:
    os.mkdir(target_source_dir)
new_image_path = target_source_dir + '/' + random_image_name
cv2.imwrite(new_image_path, cv2.imread(full_image_path))

True

In [23]:
# MAIN_PROCESS: Load saved images and classify to leaves classes

In [24]:
def load_and_preprocess_image(path):
#     check if image extension is not 'png', like 'jpg' or something else ... (only 'jpg' tested, others are not)
    if (path.split('.')[-1] != 'png'):
        print(1)
        img = Image.open(path)
        img = np.array(img).reshape(img.size[0], img.size[1], 3)
    else:
        print(2)
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=3)
    image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image_blur = cv2.GaussianBlur(image_gray, (3, 3), 0)
    image_border_only = cv2.Canny(image=image_blur, threshold1 = 100, threshold2 = 200)
    img = tf.image.resize(img, (300, 300))
    
#     print(img)
    return img
# load_and_preprocess_image(new_image_path)

In [25]:
target_image_path = new_image_path # change 'new_image_path' to saved image's path
print(target_image_path) # just to check predicted class

image = [load_and_preprocess_image(target_image_path)]
DS_test = tf.data.Dataset.from_tensor_slices(image)
DS_batch_test = DS_test.batch(batch_size=1, drop_remainder=False)
model = load_model('model.h5')
result = model.predict(DS_batch_test, batch_size=1, verbose=1, max_queue_size=1)
print(result)


./images/waiting_for_classification/IMG_20221025_180229_927.jpg
1
1/1 [==============================] - 0s 203ms/step
[[1.3784766e-17 1.5580496e-13 1.0000000e+00]]


In [26]:
def load_file(path):
    file = open(path, 'r')
    content = file.read()
    file.close()
    return content
file_path = 'class_list.txt'
content = load_file(file_path)
class_list = content.split(' ')
print('Predicted Result: ' + class_list[np.argmax(result)])

Predicted Result: tia_to
